## Bird Species Identification with LLMs and BirdNET MCP 🐦

### Overview 🦆
This notebook uses an MCP (Model Context Protocol) server to enable LLMs to interact with [BirdNET](https://birdnet.cornell.edu) for bird species identification.  BirdNET was developed by Cornell University's Lab of Ornithology and can identify over 6,000 bird species worldwide from their vocalizations.

### MCP Architecture 🦜
The BirdNET MCP server exposes these core functions as tools:
+ `predict_species_within_audio_files_mp()` - Batch process multiple files
+ `location_based_prediction()` - Get species predictions based on geographic location and time

### Requirements 🦉
+ Python 3.11 or lower (required by `birdnet` package)
+ To run on CyVerse: Use `Jupyter_Lab_PyTorch_CPU` VICE app. Tested with 8 CPU Cores, 64GB min memory, 128 min disk space
+ MCP server implementation (`birdnet_mcp_server.py`)
+ LLM API access (Provided through AI Verde: [https://chat.cyverse.ai](https://chat.cyverse.ai))
---------

## Model Context Protocol (MCP)
+ Open-source protocol by Anthropic (Nov 2024) to standardize how AI systems interact with external data sources and tools
+ Allows LLMs to:
    + Connect to data sources (databases, files, APIs)
    + Call tools (functions the LLM can execute)
    + Tools can make API calls, run software, search data, etc.
+ Server/Client architecture:
    + Server: Defines and exposes tools/resources
    + Client: AI application that connects to servers
    + LLM: Decides when to use which tools autonomously
    + Python packages like `fastmcp` and `mcp_use` simplify building custom MCP servers and clients

![MCP Architecture](https://cdn.sanity.io/images/599r6htc/regionalized/da4205446e7c425053653d58c5aed6fac556c659-2160x1440.png?q=75&fit=max&auto=format&dpr=2)

### Dataset

Audio recordings are located in the `data` folder.  Use the following cell to listen to the recordings!

File names for easy copy/paste if you'd like to test different audio files:

+ XC358703-lesser-goldfinch.mp3
+ XC436528-black-tailed-gnatcatcher.mp3
+ XC528331-curve-billed-thrasher.mp3
+ XC544598-verdin.mp3
+ XC578664-rufous-winged-sparrow.mp3
+ XC589950-gambels-quail.mp3
+ XC66350-costas-hummingbird.mp3
+ XC71035-broad-billed-hummingbird.mp3
+ XC75502-annas-hummingbird.mp3
+ XC767284-great-horned-owl.mp3
+ XC872133-mourning-dove.mp3
+ XC950378-aberts-towhee.wav
+ XC982241-vermillion-flycatcher.wav

In [1]:
from IPython.display import Audio

# From a file
Audio('./data/XC358703-lesser-goldfinch.mp3')


### Install required packages:
- `langchain_openai` -  Wrapper for calling LLMs via OpenAI-compatible APIs
- `birdnet` - Bird species identification model
- `librosa`, `soundfile` - Audio processing libraries
- `fastmcp` - MCP server framework
- `mcp_use` - MCP client library that connects LLMs to MCP tools

In [ ]:
!pip install -q langchain_openai==0.3.34
!pip install -q birdnet==0.1.7 librosa==0.11.0 soundfile==0.13.1
!pip install -q fastmcp==2.12.4 mcp_use==1.3.11 asyncio==4.0.0 

### Go to AI Verde
#### Let's take a quick tour and grab our API Key while we're at it:
[https://chat.cyverse.ai](https://chat.cyverse.ai).

### Add your API Key

When prompted below, enter your API Key from AI Verde:

In [ ]:
from getpass import getpass

api_key = getpass('Enter your API key & hit enter: ')

### Get listing of available models

The following `curl` command lists the available models for your API key:

In [ ]:
!curl -s -L "https://llm-api.cyverse.ai/v1/models" \
  -H "Authorization: Bearer {api_key}" \
  -H 'Content-Type: application/json' | json_pp

### Initialize LLM

The following initializes the LLM from AI Verde  (in this case, `llama-4-scout` hosted by Jetstream 2, but feel free to try another model from the list!)

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="js2/llama-4-scout",
    api_key=api_key,
    base_url="https://llm-api.cyverse.ai/v1"
)

Now we can test the LLM using `llm.invoke()`:

In [ ]:
print (llm.invoke("Hello, world!"))

### Run the MCP Server

We can run the MCP server directly from the notebook as a subprocess OR we can open a terminal and run it as its own process using `python birdnet_mcp_server.py` (from the same directory as the notebook).  The following cell runs it as a subprocess to keep everything in the notebook:

In [ ]:
import subprocess
import time

# Start the server as a background process
server_process = subprocess.Popen(
    ["python", "birdnet_mcp_server.py"],
)

# Wait a moment for the server to start up
time.sleep(5) 
print("MCP Server started in background on port 8000.")

### Initialize the MCP client

Connect to the BirdNET MCP server running on port 8000. The client enables the LLM to discover and call available tools.

In [ ]:
from mcp_use import MCPClient

client = MCPClient.from_dict({
    "mcpServers": {
        "birdnet_server": {
            "url": "http://127.0.0.1:8000/mcp"
            # "command": "python", 
            # "args": ["./birdnet_mcp_server.py"],
            # "transport": "stdio" 
        }
    }
})

### Set up the agent and run with prompt

The following function executes the MCP agent with a user prompt. The agent connects the LLM to MCP tools and orchestrates the workflow:
1. **Agent** receives your prompt and gives the LLM access to BirdNET tools via MCP
2. **LLM** reads the prompt, decides which BirdNET functions to call, and requests them
3. **Agent** routes the tool calls through the MCP Client to the BirdNET MCP Server
4. **BirdNET MCP Server** executes the analysis and returns results
5. **Agent** feeds results back to the LLM, which interprets them into a natural language response

**Parameters:**
- `prompt`: Question or instruction about bird species identification
- `max_steps`: Maximum tool calls the agent can make (default: 5)
- `llm_instance`: Optional LLM to use, defaults to global `llm` if not specified

In [ ]:
from mcp_use import MCPAgent

async def run_agent_with_prompt(
    prompt: str, 
    max_steps: int = 5,
    llm_instance = None
):
    # Use provided LLM or fall back to global
    active_llm = llm_instance if llm_instance is not None else llm
    
    agent = MCPAgent(llm=active_llm, client=client, max_steps=max_steps)
    
    print(f"User Prompt: {prompt}\n")
    print("--- Agent Execution ---")
    
    result = await agent.run(prompt)
    
    print("\n--- Final Answer ---")
    print(result)
    
    return result

### Experiment with different prompts

Add or modify prompts in the `TEST_PROMPTS` dictionary to experiment with different questions.

**Usage:**
- Run a test prompt: `await run_agent_with_prompt(TEST_PROMPTS["single_file"])`
- Run a custom prompt: `await run_agent_with_prompt("Your custom question here")`

File names for easy copy/paste if you'd like to test different audio files:

+ XC358703-lesser-goldfinch.mp3
+ XC436528-black-tailed-gnatcatcher.mp3
+ XC528331-curve-billed-thrasher.mp3
+ XC544598-verdin.mp3
+ XC578664-rufous-winged-sparrow.mp3
+ XC589950-gambels-quail.mp3
+ XC66350-costas-hummingbird.mp3
+ XC71035-broad-billed-hummingbird.mp3
+ XC75502-annas-hummingbird.mp3
+ XC767284-great-horned-owl.mp3
+ XC872133-mourning-dove.mp3
+ XC950378-aberts-towhee.wav
+ XC982241-vermillion-flycatcher.wav


In [ ]:
# Test prompts
TEST_PROMPTS = {
    "single_file": "What bird species can be heard in ./data/XC589950-gambels-quail.mp3?",
    "with_confidence": "Analyze ./data/XC589950-gambels-quail.mp3 and tell me which species you're most confident about.",
    "multiple_analysis": "Compare bird species in ./data/XC589950-gambels-quail.mp3 and ./data/XC75502-annas-hummingbird.mp3",
    "location_context": "What birds are typically found in Arizona based on the current date and location 32.2319° N, 110.9501° W?",
}

# Usage examples (uncomment to use):

# 1. Run a single prompt by key
await run_agent_with_prompt(TEST_PROMPTS["single_file"])

# 2. Run a custom prompt
# await run_agent_with_prompt("Compare bird species in ./data/XC589950-gambels-quail.mp3 and ./data/XC75502-annas-hummingbird.mp3")

# 3. Run a specific test prompt with confidence
# result = await run_agent_with_prompt(TEST_PROMPTS["with_confidence"])

### Experiment with different LLMs

Different LLMs have varying capabilities for tool use with MCP:
- Some models may struggle to call tools correctly and consistently or call them unnecessarily
- Others may fail to call tools at all even when needed

**Performance depends on:**
- The model's training for function calling & ability to interpret tool schemas  
- **Tool schema quality** - clear descriptions, detailed parameter explanations, and examples in the `@mcp.tool()` decorator help the LLM understand when and how to use tools correctly

In [ ]:
# Instantiate another LLM
alt_llm = ChatOpenAI(
    model="anvilgpt/gemma:latest",
    api_key=api_key,
    base_url="https://llm-api.cyverse.ai/v1"
)

# 3. Compare results from different LLMs
llm_result = await run_agent_with_prompt(TEST_PROMPTS["single_file"])
alt_llm_result = await run_agent_with_prompt(TEST_PROMPTS["single_file"], llm_instance=alt_llm)

### Challenge Exercises 🦅🐦

1. Get LLM to successfully call `predict_species_by_location` MCP tool.  What model did you use? Did you have to modify the prompt or the tool itself?
2. Add MCP tools to allow LLM to get the current date, and latitude and longitude for a city, so the user doesn't have to provide this information when predicting species by location/time of year.  Otherwise the LLM just makes stuff up for these values! 🚨